In [18]:
import pandas as pd
import requests
import string
string.punctuation
import re
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import ConfusionMatrixDisplay


from sklearn.model_selection import(
    cross_val_score,
    train_test_split,
    GridSearchCV
)

from sklearn.preprocessing import (
    StandardScaler,
    PolynomialFeatures
)


pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Pre-processing

In [19]:
df_anxiety = pd.read_csv('Anxiety.csv')
df_ptsd = pd.read_csv('PTSD.csv')

In [20]:
df_anxiety = df_anxiety.drop(columns = 'Unnamed: 0')
df_ptsd.drop(columns = 'Unnamed: 0', inplace = True)

In [21]:
df = pd.concat([df_ptsd,df_anxiety])
df.reset_index(inplace = True)
df.drop(columns = 'index', inplace = True)

In [22]:
df.shape

(20096, 5)

In [23]:
# define function to remove punctuation and remove \n
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    punctuationfree = ' '.join([sentence for sentence in punctuationfree.split('\n') if sentence != ''])
    return punctuationfree

columns = ['selftext', 'title']

for col in columns:
    new_col = str('clean_' + col)
    df[new_col] = df[col].apply(lambda x: remove_punctuation(x))   

In [ ]:
# changing all to lower caps 

columns = ['clean_selftext', 'clean_title']

for col in columns:
    new_col = str('lower_' + col.split('_')[1])
    df[new_col] = df[col].apply(lambda x: x.lower())   

In [ ]:
# split into single words 

columns = ['lower_selftext', 'lower_title']

for col in columns:
    new_col = str('split_' + col.split('_')[1])
    df[new_col] = df[col].apply(lambda x: x.split())   

In [ ]:
#Stop words present in the library
stopwords = nltk.corpus.stopwords.words('english')
stopwords[0:10]

#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

columns = ['split_selftext', 'split_title']

for col in columns:
    new_col = str('stopwords_' + col.split('_')[1])
    df[new_col] = df[col].apply(lambda x:remove_stopwords(x))   

In [ ]:
#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

#defining the function for lemmatization
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text

columns = ['stopwords_selftext', 'stopwords_title']

for col in columns:
    new_col = str('lemmatized_' + col.split('_')[1])
    df[new_col] = df[col].apply(lambda x:lemmatizer(x))   

In [ ]:
df.head(1)

,subreddit,selftext,title,created_utc,datetime,clean_selftext,clean_title,lower_selftext,lower_title,split_selftext,split_title,stopwords_selftext,stopwords_title,lemmatized_selftext,lemmatized_title
0,Anxiety,"after years of making my own vlogs and digital content and keeping it private and over 14 years of writing poems and overthinking about what others will think if i ever shared one, i finally posted my first poem online and for the public to see and criticize if it comes to it. it would mean the world if anyone checked it out and even had some comments or tips to make. \nhttps://vm.tiktok.com/TTPdPSsW7g/\n\ni’m also trying to post everyday bc who tf cares if someone i know hates what im doing or thinks i suck at what im trying to do. i’ll learn along the way and although my anxiety stays by my side i’m proud of myself for posting today and using my own voice and sharing my own poem. criticism can be scary and its scary thinking “this must be so embarrassing” but fck it. just fvk it. the more time that goes on the more i realize how small we are and how we dont know anyone and no one knows us so why do we care so much about what they think!! fck it!! x3 x100!!! just keep saying it and let it push you towards that first step you’re too afraid to take.",fck what others think! trying to shove my anxiety aside,1647751966,2022-03-20 04:52:46,after years of making my own vlogs and digital content and keeping it private and over 14 years of writing poems and overthinking about what others will think if i ever shared one i finally posted my first poem online and for the public to see and criticize if it comes to it it would mean the world if anyone checked it out and even had some comments or tips to make httpsvmtiktokcomTTPdPSsW7g i’m also trying to post everyday bc who tf cares if someone i know hates what im doing or thinks i suck at what im trying to do i’ll learn along the way and although my anxiety stays by my side i’m proud of myself for posting today and using my own voice and sharing my own poem criticism can be scary and its scary thinking “this must be so embarrassing” but fck it just fvk it the more time that goes on the more i realize how small we are and how we dont know anyone and no one knows us so why do we care so much about what they think fck it x3 x100 just keep saying it and let it push you towards that first step you’re too afraid to take,fck what others think trying to shove my anxiety aside,after years of making my own vlogs and digital content and keeping it private and over 14 years of writing poems and overthinking about what others will think if i ever shared one i finally posted my first poem online and for the public to see and criticize if it comes to it it would mean the world if anyone checked it out and even had some comments or tips to make httpsvmtiktokcomttpdpssw7g i’m also trying to post everyday bc who tf cares if someone i know hates what im doing or thinks i suck at what im trying to do i’ll learn along the way and although my anxiety stays by my side i’m proud of myself for posting today and using my own voice and sharing my own poem criticism can be scary and its scary thinking “this must be so embarrassing” but fck it just fvk it the more time that goes on the more i realize how small we are and how we dont know anyone and no one knows us so why do we care so much about what they think fck it x3 x100 just keep saying it and let it push you towards that first step you’re too afraid to take,fck what others think trying to shove my anxiety aside,"[after, years, of, making, my, own, vlogs, and, digital, content, and, keeping, it, private, and, over, 14, years, of, writing, poems, and, overthinking, about, what, others, will, think, if, i, ever, shared, one, i, finally, posted, my, first, poem, online, and, for, the, public, to, see, and, criticize, if, it, comes, to, it, it, would, mean, the, world, if, anyone, checked, it, out, and, even, had, some, comments, or, tips, to, make, httpsvmt

## Finalising Data

In [ ]:
df = df[['subreddit', 'lemmatized_selftext', 'lemmatized_title', 'datetime']]

In [ ]:
df.head()

,subreddit,lemmatized_selftext,lemmatized_title,datetime
0,Anxiety,"[year, making, vlogs, digital, content, keeping, private, 14, year, writing, poem, overthinking, others, think, ever, shared, one, finally, posted, first, poem, online, public, see, criticize, come, would, mean, world, anyone, checked, even, comment, tip, make, httpsvmtiktokcomttpdpssw7g, i’m, also, trying, post, everyday, bc, tf, care, someone, know, hate, im, think, suck, im, trying, i’ll, learn, along, way, although, anxiety, stay, side, i’m, proud, posting, today, using, voice, sharing, poem, criticism, scary, scary, thinking, “this, must, embarrassing”, fck, fvk, time, go, realize, small, dont, know, anyone, one, know, u, care, much, think, fck, x3, x100, keep, saying, let, push, towards, first, step, ...]","[fck, others, think, trying, shove, anxiety, aside]",2022-03-20 04:52:46
1,Anxiety,"[crush, work, together, campus, restaurant, register, two, guy, walked, got, food, paid, cash, wasnt, paying, attention, accidentally, forgot, close, register, realized, soon, crush, overheard, happened, restocking, can, walked, asked, happened, told, happened, tried, see, could, unlock, machine, need, id, number, open, didnt, work, one, else, working, needed, manager, open, manager, lunch, needed, clock, back, open, whole, ordeal, took, 20, minute, felt, bad, crush, helpful, apologized, whole, thing, kind, know, big, deal, moment, shaken, felt, bad, two, guy, wait, crush, getting, dragged, mess, manager, get, lunch, early, really, shaken, im, normally, shy, person, much, timid, wasnt, talkative, didnt, really, want, look, ...]","[anxiety, resurfaced, crush, helped]",2022-03-20 04:44:10
2,Anxiety,"[hi, im, wondering, anyone, issue, tip, cope, ive, night, club, twice, past, couple, month, ive, drink, friend, occasion, ive, leave, early, get, anxious, number, people, around, resulting, panic, attack, never, used, like, covid, im, generally, ok, crowd, sober, anyone, got, tip, cope]","[anxiety, drinking, following, covid, lockdown]",2022-03-20 04:43:48
3,Anxiety,"[feeling, shitty, everyone, age, already, got, life, together, still, dont, shit, driving, license, part, community, friend, count, nothing, im, completely, alone, world, ive, wasting, much, time, throughout, whole, life, concerning, much, academic, yet, still, dumb, incapable, anything, stupid, anxiety, get, nowhere, life, cant, even, make, friend, got, feeling, nobody, want, friend, would, cant, even, talk, anyone, without, anxious, cant, tell, anyone, believe, nobody, care, waste, time, there, future, im, shitty, totally, worthless]",[suck],2022-03-20 04:36:18
4,Anxiety,"[im, 18, ive, plantar, wart, almost, 3, year, im, worried, carelessness, might, spread, dick, shower, know, almost, strain, cause, plantars, one, infect, genitals, type, 66, one, disease, share, doom, acrolling, something, im, freaking, bit, reading, treatment, wart, fuck, dick, reading, risk, dont, take, treatment, getting, depressed, angry, treating, issue, sooner]","[needing, help, health, anxiety, attack]",2022-03-20 04:26:25


In [ ]:
X = df.drop(columns = 'subreddit')
y = df['subreddit']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Apply standard scaler

ss = StandardScaler()
ss.fit(X_train,y_train)

ValueError: setting an array element with a sequence.

In [ ]:
# Declare KNN

knn = KNeighborsClassifier()

knn.fit()